# Clean Excel data from Dalinde Hospital

In [1]:
import pandas as pd
import matplotlib as plt
import numpy as np
import sklearn
import os
from tqdm.auto import tqdm, trange

_________________

## Informes de consumo 2018-2019 (PatientConsumption)

In [2]:
# Retrieve names of files
data_files = os.listdir('data/')
data_files = [file for file in data_files if 'Informe de consumo por areas Tuxpan' in file]
data_files

['Informe de consumo por areas Tuxpan enero 2018.xlsx',
 'Informe de consumo por areas Tuxpan Febrero 2018.xlsx',
 'Informe de consumo por areas Tuxpan Diciembre 2018.xlsx',
 'Informe de consumo por areas Tuxpan Agosto 2019.xlsx',
 'Informe de consumo por areas Tuxpan Septiembre 2019.xlsx',
 'Informe de consumo por areas Tuxpan Noviembre 2018.xlsx',
 'Informe de consumo por areas Tuxpan Junio 2019.xlsx',
 'Informe de consumo por areas Tuxpan mayo 2018.xlsx',
 'Informe de consumo por areas Tuxpan Mayo 2019.xlsx',
 'Informe de consumo por areas Tuxpan Marzo 2019.xlsx',
 'Informe de consumo por areas Tuxpan Noviembre 2019.xlsx',
 'Informe de consumo por areas Tuxpan Julio 2019.xlsx',
 'Informe de consumo por areas Tuxpan Septiembre 2018.xlsx',
 'Informe de consumo por areas Tuxpan marzo 2018.xlsx',
 'Informe de consumo por areas Tuxpan junio 2018.xlsx',
 'Informe de consumo por areas Tuxpan Julio 2018.xlsx',
 'Informe de consumo por areas Tuxpan Octubre 2019.xlsx',
 'Informe de consumo po

In [3]:
# Months dictionary
months = {'ENERO':'01', 'FEBRERO':'02', 'MARZO':'03', 'ABRIL':'04', 'MAYO':'05', 'JUNIO':'06', 'JULIO':'07', 'AGOSTO':'08', 'SEPTIEMBRE':'09', 'OCTUBRE':'10', 'NOVIEMBRE':'11', 'DICIEMBRE':'12'}
# Empty DataFrame
data = pd.DataFrame()

for i in trange(len(data_files), desc = 'Processing files'):
      
    filename = data_files[i] # Name of file
    
    # Retreive MONTH and YEAR from filename
    month = months[filename.split(' ')[6].upper()]
    year = filename.split(' ')[7].split('.')[0]
    # Set date with first day of the month
    date = pd.to_datetime(year+month, format='%Y%m')
    # Assign last day of the month
    date = pd.to_datetime(year+month+str(date.days_in_month), format='%Y%m%d')
    
    print('Processing data from '+str(date)) # For tracking progress
    
    # Load file
    df = pd.read_excel('data/'+filename, header=5)
    
    # Fill ÁREA and DEPARTAMENTO columns since the only appear once
    df[["ÁREA","DEPARTAMENTO"]] = df[["ÁREA","DEPARTAMENTO"]].fillna(method='ffill')
    # Drop all rows without a CAPTURA id, which are thrash 
    df = df.loc[df['CAPTURA'].dropna().index, :]
    # Retrieve items which DEPARTAMENTO =='PATENTE'
    df = df.loc[df['DEPARTAMENTO'] == 'PATENTE', :]
    # Add FECHA column
    df["FECHA"] = date
    df = df.loc[:, ["FECHA", "DEPARTAMENTO", "CAPTURA", "CÓDIGO", "ARTÍCULO", "CANTIDAD", "COSTO UNITARIO","TOTAL NETO"]]
    
    # Concatenate df
    data = pd.concat([data, df], ignore_index=True)

# Sort values by date
data = data.sort_values(['FECHA','ARTÍCULO'], ignore_index=True)

# Rename columns to standard names
data = data.rename(columns={
    "FECHA":"ConsumptionDate",
    "DEPARTAMENTO":"Department",
    "CAPTURA":"ConsumptionNumber",
    "CÓDIGO":"MedCode",
    "ARTÍCULO":"MedDescription",
    "CANTIDAD":"AmmountConsumed",
    "COSTO UNITARIO":"UnitaryCost",
    "TOTAL NETO":"TotalCost"
})

# Write data to CSV file
data.to_csv('PatientConsumption 2018-2019.csv', index=False)
data

Processing data from 2018-01-31 00:00:00
Processing data from 2018-02-28 00:00:00
Processing data from 2018-12-31 00:00:00
Processing data from 2019-08-31 00:00:00
Processing data from 2019-09-30 00:00:00
Processing data from 2018-11-30 00:00:00
Processing data from 2019-06-30 00:00:00
Processing data from 2018-05-31 00:00:00
Processing data from 2019-05-31 00:00:00
Processing data from 2019-03-31 00:00:00
Processing data from 2019-11-30 00:00:00
Processing data from 2019-07-31 00:00:00
Processing data from 2018-09-30 00:00:00
Processing data from 2018-03-31 00:00:00
Processing data from 2018-06-30 00:00:00
Processing data from 2018-07-31 00:00:00
Processing data from 2019-10-31 00:00:00
Processing data from 2018-08-31 00:00:00
Processing data from 2019-12-31 00:00:00
Processing data from 2019-04-30 00:00:00
Processing data from 2019-01-31 00:00:00
Processing data from 2018-04-30 00:00:00
Processing data from 2019-02-28 00:00:00
Processing data from 2018-10-31 00:00:00



,ConsumptionDate,Department,ConsumptionNumber,MedCode,MedDescription,AmmountConsumed,UnitaryCost,TotalCost
0,2018-01-31,PATENTE,5235410.0,3300629.0,ACC 200MG C/20 TAB EFERV (ACETILCISTEINA),1.0,127.59,127.59
1,2018-01-31,PATENTE,5233949.0,3304626.0,ACC 600MG TAB EFERV (ACETILCISTEINA),1.0,252.48,252.48
2,2018-01-31,PATENTE,5235566.0,3304626.0,ACC 600MG TAB EFERV (ACETILCISTEINA),1.0,252.48,252.48
3,2018-01-31,PATENTE,5245250.0,3302703.0,ACELER CO 2% CREMA 15G(ACIDO FUSIDICO/BETAMETA...,1.0,206.73,206.73
4,2018-01-31,PATENTE,5265293.0,3302703.0,ACELER CO 2% CREMA 15G(ACIDO FUSIDICO/BETAMETA...,1.0,206.73,206.73
...,...,...,...,...,...,...,...,...
100302,2019-12-31,PATENTE,6183131.0,3300140.0,ZYPREXA IM 10MG AMP 2ML (OLANZAPINA),1.0,292.56,292.56
100303,2019-12-31,PATENTE,6173743.0,3300140.0,ZYPREXA IM 10MG AMP 2ML (OLANZAPINA),1.0,292.56,292.56
100304,2019-12-31,PATENTE,6165542.0,3300182.0,ZYPREXA ZYDIS 10MG C/14 TABS (OLANZAPINA),1.0,1710.24,1710.24
100305,2019-12-31,PATENTE,6201334.0,3600131.0,ZYVOXAM TABLETAS 600MG C/10 (LINEZOLID),1.0,7607.83,7607.83


_________________

## Requisiciones vs órdenes de compra (PurchasedOrders)

In [4]:
# Retrieve names of files
data_files = os.listdir('data/')
data_files = [file for file in data_files if 'requisiciones vs ordenes de compra' in file]
data_files

['requisiciones vs ordenes de compra 2019.xlsx',
 'requisiciones vs ordenes de compra 2018 (1).xlsx']

In [5]:
# Empty DataFrame
data = pd.DataFrame()

for i in trange(len(data_files), desc = 'Processing files'):

    filename = data_files[i] # Name of file
    print('Processing data from '+filename) # For tracking progress
    
    # Load file
    df = pd.read_excel('data/'+filename, header=[4,5])
    # Restore columns names
    df.columns = [(col[0] if 'Unnamed' in col[1] else col[0]+' '+col[1]) for col in df.columns.to_flat_index()]
    df = df.rename(columns={'DESCRIPCIÓN SALIDAS':'SALIDAS'})
    
    # Concatenate df
    data = pd.concat([data, df], ignore_index=True)

# Drop unused columns
data = data.loc[:, data.columns[:-2]]
# Standirize date format
data[['FECHA','FECHA PEDIDO','FECHA ENTREGA']] = data[['FECHA','FECHA PEDIDO','FECHA ENTREGA']].apply(pd.to_datetime, format='%d-%m-%Y')

# Sort values by REQUISICIÓN id
data = data.sort_values(['REQUISICIÓN','FECHA'], ignore_index=True)

# Rename columns to standard names
data = data.rename(columns={
    "REQUISICIÓN":"RequestNumber",
    "FECHA":"RequestDate",
    "ORDEN COMPRA":"PurchaseNumber",
    "FECHA PEDIDO":"OrderDate",
    "FECHA ENTREGA":"RequiredDeliveryDate",
    "PROVEEDOR":"SupplierName",
    "ARTÍCULO":"MedCode",
    "DESCRIPCIÓN":"MedDescription",
    "SALIDAS":"Outflow",
    "CANTIDAD REQUERIDA":"AmountRequested",
    "CANTIDAD PEDIDA":"AmountPurchased"
})

# Write data to CSV file
data.to_csv('PurchasedOrders 2018-2019.csv', index=False)
data

Processing data from requisiciones vs ordenes de compra 2019.xlsx
Processing data from requisiciones vs ordenes de compra 2018 (1).xlsx



,RequestNumber,RequestDate,PurchaseNumber,OrderDate,RequiredDeliveryDate,SupplierName,MedCode,MedDescription,Outflow,AmountRequested,AmountPurchased
0,52666,2018-01-02,257941.0,2018-01-03,2018-01-10,"NADRO, S.A.P.I. DE C.V.",2900110,#DEXERYL CREMA EMOLIENTE 250G,199.0,1.0,1
1,52666,2018-01-02,257941.0,2018-01-03,2018-01-10,"NADRO, S.A.P.I. DE C.V.",2900233,#ISODINE 8G BUCOFARINGEO SOL 120ML,198.0,2.0,2
2,52666,2018-01-02,257941.0,2018-01-03,2018-01-10,"NADRO, S.A.P.I. DE C.V.",2900274,#VASELINE PURO 100G,196.0,2.0,2
3,52666,2018-01-02,257941.0,2018-01-03,2018-01-10,"NADRO, S.A.P.I. DE C.V.",2900493,VIVERA 2G 8 SOBRES LACTOBACILLUS RHAMNOSUS GG,197.0,2.0,2
4,52666,2018-01-02,257942.0,2018-01-03,2018-01-10,FARMACOS NACIONALES S.A. DE C.V. (FANASA),2900150,#BICARBONATO DE SODIO 100G,198.0,2.0,2
...,...,...,...,...,...,...,...,...,...,...,...
64167,66178,2019-12-30,315609.0,2019-12-30,2020-01-03,DISTRIBUIDORA MEDICA CODIMED S.A. DE C.V.,3800068,FISIOLOGICA 50 ML FCO. PISA,1282.0,1200.0,1200
64168,66178,2019-12-30,315609.0,2019-12-30,2020-01-03,DISTRIBUIDORA MEDICA CODIMED S.A. DE C.V.,3800066,FISIOLOGICA 500ML FCO. PISA,166.0,168.0,168
64169,66178,2019-12-30,315609.0,2019-12-30,2020-01-03,DISTRIBUIDORA MEDICA CODIMED S.A. DE C.V.,3800064,HARTMAN 1000ML FRASCO PISA,143.0,120.0,120
64170,66178,2019-12-30,315609.0,2019-12-30,2020-01-03,DISTRIBUIDORA MEDICA CODIMED S.A. DE C.V.,3800077,HARTMAN 500 ML FLEXOVAL SOLUCION PISA 4000088,78.0,72.0,72


_________________

## Informe órdenes incompletas (IncompleteOrders)

In [6]:
# Retrieve names of files
data_files = os.listdir('data/')
data_files = [file for file in data_files if 'Requisiciones ordenes incompletas' in file]
data_files

['Requisiciones ordenes incompletas 2018.xlsx',
 'Requisiciones ordenes incompletas 2019.xlsx']

In [7]:
# Empty DataFrame
data = pd.DataFrame()

for i in trange(len(data_files), desc = 'Processing files'):

    filename = data_files[i] # Name of file
    print('Processing data from '+filename) # For tracking progress
    
    # Load file
    df = pd.read_excel('data/'+filename, header=[4,5])
    # Restore columns names
    df.columns = [(col[0] if 'Unnamed' in col[1] else col[0]+' '+col[1]) for col in df.columns.to_flat_index()]
    
    # Concatenate df
    data = pd.concat([data, df], ignore_index=True)

# Drop unused columns
data = data.loc[:, np.append(data.columns[:-3], data.columns[-2])]
# Standirize date format
data[['FECHA PEDIDO','FECHA ENTREGA','FECHA SOLICITUD','FECHA RECEPCION']] = data[['FECHA PEDIDO','FECHA ENTREGA','FECHA SOLICITUD','FECHA RECEPCION']].apply(pd.to_datetime, format='%d-%m-%Y', errors='coerce')
data

Processing data from Requisiciones ordenes incompletas 2018.xlsx
Processing data from Requisiciones ordenes incompletas 2019.xlsx



,PROVEEDOR,PEDIDO,FECHA PEDIDO,FECHA ENTREGA,REQUISICION,FECHA SOLICITUD,C. COSTOS,ARTÍCULO,DESCRIPCIÓN,FACTURA,FECHA RECEPCION,CANTIDAD REQUISICION,CANTIDAD SOLICITADA,CANTIDAD RECIBIDA,%
0,"ABASTECEDOR TERAPEUTICO, S.A. DE C.V. (ATSA)",258679.0,2018-01-15,2018-01-22,52889.0,2018-01-15,MATERIAL DE CURACION,1900240,#EXTENSION K/750 50CM MEDEX MX450FL,NaN,NaT,250.0,250.0,0,0.0
1,"ABASTECEDOR TERAPEUTICO, S.A. DE C.V. (ATSA)",262080.0,2018-02-26,2018-03-05,53711.0,2018-02-26,MATERIAL DE CURACION,1900947,#EXTENSION MINIVOLUMEN 536035 1.0ML VOL RESIDU...,NaN,NaT,50.0,50.0,0,0.0
2,NaN,NaN,NaT,NaT,NaN,NaT,NaN,1900979,#KIT P/MONITOREO MX9505T TRANSDUCTOR MEDEX,NaN,NaT,10.0,10.0,0,0.0
3,NaN,NaN,NaT,NaT,NaN,NaT,NaN,1901413,#KIT P/MONITOREO MX9604A TRANSDUCTOR MEDEX,NaN,NaT,20.0,20.0,0,0.0
4,"ABASTECEDOR TERAPEUTICO, S.A. DE C.V. (ATSA)",268297.0,2018-05-21,2018-05-28,55285.0,2018-05-21,MATERIAL DE CURACION,1900313,#LLAVE 3 VIAS S/EXTENSION MX5311L MEDEX,NaN,NaT,250.0,250.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7105,"VITASANITAS, S.A. DE C.V.",303167.0,2019-08-06,2019-08-07,63296.0,2019-08-06,MEDICAMENTOS,4200011,SMOF KABIVEN 1100KCAL CENTRAL (TRICAMARA NUTRC...,NaN,NaT,2.0,2.0,0,0.0
7106,"VITASANITAS, S.A. DE C.V.",309257.0,2019-10-18,2019-10-21,0.0,NaT,MEDICAMENTOS,4200011,SMOF KABIVEN 1100KCAL CENTRAL (TRICAMARA NUTRC...,NaN,NaT,0.0,10.0,0,0.0
7107,"VITASANITAS, S.A. DE C.V.",309763.0,2019-10-23,2019-10-24,64862.0,2019-10-23,MEDICAMENTOS,4200011,SMOF KABIVEN 1100KCAL CENTRAL (TRICAMARA NUTRC...,NaN,NaT,2.0,2.0,0,0.0
7108,"VITASANITAS, S.A. DE C.V.",310438.0,2019-11-01,2019-11-04,65015.0,2019-11-01,MEDICAMENTOS,3800003,ALBUMINA HUMANA AL 20% FCO 50ML,NaN,NaT,6.0,6.0,0,0.0


There seems to be many Na values. Let's count them.

In [8]:
data.isna().sum()

PROVEEDOR               4761
PEDIDO                  4761
FECHA PEDIDO            4761
FECHA ENTREGA           4761
REQUISICION             4761
FECHA SOLICITUD         5207
C. COSTOS               4761
ARTÍCULO                   0
DESCRIPCIÓN                0
FACTURA                 5640
FECHA RECEPCION         5640
CANTIDAD REQUISICION     637
CANTIDAD SOLICITADA      637
CANTIDAD RECIBIDA          0
%                          0
dtype: int64

Only ARTÍCULO, DESCRIPCIÓN, CANTIDAD RECIBIDA and % have no missing values. This is a good sign that the data is not corrupted.<br>
After exploring the data we concluded that we should not fill any Na values in the FACTURA and FECHA RECEPCION columns, as they serve as an indicator for orders that were not received.<br>
We also spotted the pattern that all rows with 'REQUISICION'==0 are missing 'FECHA SOLICITUD'. We most explore this further in the data analysis.

In [9]:
data[data['REQUISICION']==0]['FECHA SOLICITUD'].value_counts()

Series([], Name: FECHA SOLICITUD, dtype: int64)

As mentioned before, FACTURA and FECHA RECEPCION columns serve as an indicator for orders that were not received. We confirm it by noticing that fulfillment ('%') of every order missing these columns is always 0%.

In [10]:
data[data['FECHA RECEPCION'].isna()]['%'].value_counts()

0.0    5640
Name: %, dtype: int64

All rows that are missing 'CANTIDAD REQUISICION' and 'CANTIDAD SOLICITADA' are missing the first 7 columns too.

In [11]:
data[data['CANTIDAD REQUISICION'].isna()].isna().sum()

PROVEEDOR               637
PEDIDO                  637
FECHA PEDIDO            637
FECHA ENTREGA           637
REQUISICION             637
FECHA SOLICITUD         637
C. COSTOS               637
ARTÍCULO                  0
DESCRIPCIÓN               0
FACTURA                   0
FECHA RECEPCION           0
CANTIDAD REQUISICION    637
CANTIDAD SOLICITADA     637
CANTIDAD RECIBIDA         0
%                         0
dtype: int64

After this analysis we conclude that:<br>
1) There data for the columns ['PROVEEDOR','PEDIDO','FECHA PEDIDO','FECHA ENTREGA','REQUISICION','C. COSTOS'] only appear once per order. The consecutive rows belonging to the same order are missing these values.<br>
2) The data for the columns ['CANTIDAD REQUISICION','CANTIDAD SOLICITADA'] only appear once per article in all orders, meaning that the consecutive rows belonging to the same order and article are missing these values.<br>
3) 'FECHA SOLICITUD' also appears once per order. However, all rows with 'REQUISICION'==0 are intentionally missing 'FECHA SOLICITUD', so we should keep it like that to not corrupt the data.<br>
We will fill missing values according to the described behavours.

In [12]:
# Create a copy of the data
temp = data.copy()
# Fill in Na values from point 1)
temp[['PROVEEDOR','PEDIDO','FECHA PEDIDO','FECHA ENTREGA','REQUISICION','C. COSTOS']] = temp[['PROVEEDOR','PEDIDO','FECHA PEDIDO','FECHA ENTREGA','REQUISICION','C. COSTOS']].fillna(method='ffill')
# Fill in Na values from point 2)
temp[['CANTIDAD REQUISICION','CANTIDAD SOLICITADA']] = temp[['CANTIDAD REQUISICION','CANTIDAD SOLICITADA']].fillna(method='ffill')
# Sanity check
temp.isna().sum()

PROVEEDOR                  0
PEDIDO                     0
FECHA PEDIDO               0
FECHA ENTREGA              0
REQUISICION                0
FECHA SOLICITUD         5207
C. COSTOS                  0
ARTÍCULO                   0
DESCRIPCIÓN                0
FACTURA                 5640
FECHA RECEPCION         5640
CANTIDAD REQUISICION       0
CANTIDAD SOLICITADA        0
CANTIDAD RECIBIDA          0
%                          0
dtype: int64

In [13]:
# Fill in Na values from point 3)
temp['FECHA SOLICITUD'] = pd.concat([temp[temp['REQUISICION']!=0]['FECHA SOLICITUD'].fillna(method='ffill'), temp[temp['REQUISICION']==0]['FECHA SOLICITUD']]).sort_index()
# Sanity check
temp[temp['REQUISICION']==0].equals(temp[temp['FECHA SOLICITUD'].isna()])

True

We see that every row with 'REQUISICION'==0 is still missing 'FECHA SOLICITUD', which is what we wanted.

In [14]:
temp.isna().sum()

PROVEEDOR                  0
PEDIDO                     0
FECHA PEDIDO               0
FECHA ENTREGA              0
REQUISICION                0
FECHA SOLICITUD         3396
C. COSTOS                  0
ARTÍCULO                   0
DESCRIPCIÓN                0
FACTURA                 5640
FECHA RECEPCION         5640
CANTIDAD REQUISICION       0
CANTIDAD SOLICITADA        0
CANTIDAD RECIBIDA          0
%                          0
dtype: int64

Now that our dataset is in better shape we save it in an csv file.

In [15]:
# Rename columns to standard names
temp = temp.rename(columns={
    "PROVEEDOR":"SupplierName",
    "PEDIDO":"OrderNumber",
    "FECHA PEDIDO":"OrderDate",
    "FECHA ENTREGA":"RequiredDeliveryDate",
    "REQUISICION":"RequestNumber",
    "FECHA SOLICITUD":"RequestDate",
    "C. COSTOS":"CostCenter",
    "ARTÍCULO":"MedCode",
    "DESCRIPCIÓN":"MedDescription",
    "FACTURA":"InvoiceNumber",
    "FECHA RECEPCION":"ReceivedDate",
    "CANTIDAD REQUISICION":"AmountRequested",
    "CANTIDAD SOLICITADA":"AmountPurchased",
    "CANTIDAD RECIBIDA":"AmountReceived",
    "%":"OrderPercentageFulfillment"
})

# Write data to CSV file
temp.to_csv('IncompleteOrders 2018-2019.csv', index=False)
temp

,SupplierName,OrderNumber,OrderDate,RequiredDeliveryDate,RequestNumber,RequestDate,CostCenter,MedCode,MedDescription,InvoiceNumber,ReceivedDate,AmountRequested,AmountPurchased,AmountReceived,OrderPercentageFulfillment
0,"ABASTECEDOR TERAPEUTICO, S.A. DE C.V. (ATSA)",258679.0,2018-01-15,2018-01-22,52889.0,2018-01-15,MATERIAL DE CURACION,1900240,#EXTENSION K/750 50CM MEDEX MX450FL,NaN,NaT,250.0,250.0,0,0.0
1,"ABASTECEDOR TERAPEUTICO, S.A. DE C.V. (ATSA)",262080.0,2018-02-26,2018-03-05,53711.0,2018-02-26,MATERIAL DE CURACION,1900947,#EXTENSION MINIVOLUMEN 536035 1.0ML VOL RESIDU...,NaN,NaT,50.0,50.0,0,0.0
2,"ABASTECEDOR TERAPEUTICO, S.A. DE C.V. (ATSA)",262080.0,2018-02-26,2018-03-05,53711.0,2018-02-26,MATERIAL DE CURACION,1900979,#KIT P/MONITOREO MX9505T TRANSDUCTOR MEDEX,NaN,NaT,10.0,10.0,0,0.0
3,"ABASTECEDOR TERAPEUTICO, S.A. DE C.V. (ATSA)",262080.0,2018-02-26,2018-03-05,53711.0,2018-02-26,MATERIAL DE CURACION,1901413,#KIT P/MONITOREO MX9604A TRANSDUCTOR MEDEX,NaN,NaT,20.0,20.0,0,0.0
4,"ABASTECEDOR TERAPEUTICO, S.A. DE C.V. (ATSA)",268297.0,2018-05-21,2018-05-28,55285.0,2018-05-21,MATERIAL DE CURACION,1900313,#LLAVE 3 VIAS S/EXTENSION MX5311L MEDEX,NaN,NaT,250.0,250.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7105,"VITASANITAS, S.A. DE C.V.",303167.0,2019-08-06,2019-08-07,63296.0,2019-08-06,MEDICAMENTOS,4200011,SMOF KABIVEN 1100KCAL CENTRAL (TRICAMARA NUTRC...,NaN,NaT,2.0,2.0,0,0.0
7106,"VITASANITAS, S.A. DE C.V.",309257.0,2019-10-18,2019-10-21,0.0,NaT,MEDICAMENTOS,4200011,SMOF KABIVEN 1100KCAL CENTRAL (TRICAMARA NUTRC...,NaN,NaT,0.0,10.0,0,0.0
7107,"VITASANITAS, S.A. DE C.V.",309763.0,2019-10-23,2019-10-24,64862.0,2019-10-23,MEDICAMENTOS,4200011,SMOF KABIVEN 1100KCAL CENTRAL (TRICAMARA NUTRC...,NaN,NaT,2.0,2.0,0,0.0
7108,"VITASANITAS, S.A. DE C.V.",310438.0,2019-11-01,2019-11-04,65015.0,2019-11-01,MEDICAMENTOS,3800003,ALBUMINA HUMANA AL 20% FCO 50ML,NaN,NaT,6.0,6.0,0,0.0


_________________

## Filtering patent drugs using patent drugs diccionary

In [88]:
  # Read CSVs
PatientConsumption = pd.read_csv('PatientConsumption 2018-2019.csv')
PurchasedOrders    = pd.read_csv('PurchasedOrders 2018-2019.csv')
IncompleteOrders   = pd.read_csv('IncompleteOrders 2018-2019.csv')
PatentDrugs        = pd.read_csv('PatentDrugs.csv')

### PatientConsumption

In [89]:
PatientConsumption = PatientConsumption[PatientConsumption['MedCode'].isin(PatentDrugs['MedCode'])].reset_index(drop=True)
PatientConsumption

,ConsumptionDate,Department,ConsumptionNumber,MedCode,MedDescription,AmmountConsumed,UnitaryCost,TotalCost
0,2018-01-31,PATENTE,5235410.0,3300629.0,ACC 200MG C/20 TAB EFERV (ACETILCISTEINA),1.0,127.59,127.59
1,2018-01-31,PATENTE,5233949.0,3304626.0,ACC 600MG TAB EFERV (ACETILCISTEINA),1.0,252.48,252.48
2,2018-01-31,PATENTE,5235566.0,3304626.0,ACC 600MG TAB EFERV (ACETILCISTEINA),1.0,252.48,252.48
3,2018-01-31,PATENTE,5245250.0,3302703.0,ACELER CO 2% CREMA 15G(ACIDO FUSIDICO/BETAMETA...,1.0,206.73,206.73
4,2018-01-31,PATENTE,5265293.0,3302703.0,ACELER CO 2% CREMA 15G(ACIDO FUSIDICO/BETAMETA...,1.0,206.73,206.73
...,...,...,...,...,...,...,...,...
100302,2019-12-31,PATENTE,6183131.0,3300140.0,ZYPREXA IM 10MG AMP 2ML (OLANZAPINA),1.0,292.56,292.56
100303,2019-12-31,PATENTE,6173743.0,3300140.0,ZYPREXA IM 10MG AMP 2ML (OLANZAPINA),1.0,292.56,292.56
100304,2019-12-31,PATENTE,6165542.0,3300182.0,ZYPREXA ZYDIS 10MG C/14 TABS (OLANZAPINA),1.0,1710.24,1710.24
100305,2019-12-31,PATENTE,6201334.0,3600131.0,ZYVOXAM TABLETAS 600MG C/10 (LINEZOLID),1.0,7607.83,7607.83


In [ ]:
  # Write to CSV file
PatientConsumption.to_csv('PatientConsumption 2018-2019.csv', index=False)

### PurchasedOrders

In [90]:
temp_df = PurchasedOrders[PurchasedOrders['MedCode']=='CANCELADO']
temp_df = temp_df[temp_df['MedDescription'].isin(PatentDrugs['MedDescription'])]
temp_df['MedCode'] = temp_df.reset_index().merge(PatentDrugs, on='MedDescription').set_index('index')['MedCode_y']
temp_df

,RequestNumber,RequestDate,PurchaseNumber,OrderDate,RequiredDeliveryDate,SupplierName,MedCode,MedDescription,Outflow,AmountRequested,AmountPurchased
6544,54399,2018-04-05,264877.0,2018-04-05,2018-04-06,DISTRIBUIDORA MEDICA CODIMED S.A. DE C.V.,3305007,LIBONIDE 0.250 MG/ML FCO 2 ML (BUDESONIDA) PIEZA,717.0,60.0,60
29121,59710,2019-01-18,287425.0,2019-01-18,2019-01-22,DISTRIBUIDORA MEDICA CODIMED S.A. DE C.V.,3305181,INHEPAR 1000 UI/ML (HEPARINA),50.0,40.0,40
33783,60590,2019-03-06,NaN,NaN,NaN,NaN,3302934,COMBIVENT RESPIMAT 1.68/8.77MG C/DOSIF.(IPRATR...,1.0,2.0,0
34547,60721,2019-03-14,292041.0,2019-03-14,2019-03-15,FARMACOS NACIONALES S.A. DE C.V. (FANASA),3304193,XARELTO 15MG.C/28 COMP (RIVAROXABAN),1.0,1.0,1
35381,60904,2019-03-25,292794.0,2019-03-25,2019-03-26,"NADRO, S.A.P.I. DE C.V.",3302439,LASILACTON 50/20MG C/16 CAPS(FUROSEMIDA/ESPIRO...,1.0,1.0,1


In [91]:
PurchasedOrders.loc[temp_df.index,:] = temp_df
PurchasedOrders = PurchasedOrders[PurchasedOrders['MedCode'].astype(str).isin(PatentDrugs['MedCode'].astype(str))].reset_index(drop=True)
PurchasedOrders['MedCode'] = PurchasedOrders['MedCode'].astype(int)
print(PurchasedOrders.dtypes)
PurchasedOrders

RequestNumber             int64
RequestDate              object
PurchaseNumber          float64
OrderDate                object
RequiredDeliveryDate     object
SupplierName             object
MedCode                   int64
MedDescription           object
Outflow                 float64
AmountRequested         float64
AmountPurchased           int64
dtype: object


,RequestNumber,RequestDate,PurchaseNumber,OrderDate,RequiredDeliveryDate,SupplierName,MedCode,MedDescription,Outflow,AmountRequested,AmountPurchased
0,52670,2018-01-02,257734.0,2018-01-02,2018-01-03,DISTRIBUIDORA MEDICA CODIMED S.A. DE C.V.,3304784,BOLENTAX (ENOXAPARINA) 40 MG/0.4 ML JER PRELL C/2,15.0,5.0,6
1,52670,2018-01-02,257734.0,2018-01-02,2018-01-03,DISTRIBUIDORA MEDICA CODIMED S.A. DE C.V.,3304883,BOLENTAX (ENOXAPARINA) 60 MG/0.6 ML JER PRELL C/2,14.0,2.0,2
2,52670,2018-01-02,257734.0,2018-01-02,2018-01-03,DISTRIBUIDORA MEDICA CODIMED S.A. DE C.V.,3304284,FRESUBIN PLUS VAINILLA 236 ML. FÓRMULA ENTERAL,15.0,5.0,5
3,52670,2018-01-02,257734.0,2018-01-02,2018-01-03,DISTRIBUIDORA MEDICA CODIMED S.A. DE C.V.,3302098,NODESCRON 4MG AMPOLLETA (VECURONIO) PISA,39.0,40.0,50
4,52670,2018-01-02,257785.0,2018-01-02,2018-01-03,"NADRO, S.A.P.I. DE C.V.",3301771,SOMAZINA 1G AMP 4ML (CITICOLINA),424.0,10.0,10
...,...,...,...,...,...,...,...,...,...,...,...
26890,66154,2019-12-30,315631.0,2019-12-30,2019-12-31,FARMACOS NACIONALES S.A. DE C.V. (FANASA),3300627,NEPRO HP 236 ML VAINILLA,5.0,5.0,5
26891,66154,2019-12-30,315631.0,2019-12-30,2019-12-31,FARMACOS NACIONALES S.A. DE C.V. (FANASA),3300046,PEDIALYTE SR-45 500ML,2.0,2.0,2
26892,66154,2019-12-30,315631.0,2019-12-30,2019-12-31,FARMACOS NACIONALES S.A. DE C.V. (FANASA),3301771,SOMAZINA 1G AMP 4ML (CITICOLINA),13.0,10.0,10
26893,66156,2019-12-30,315563.0,2019-12-30,2019-12-31,LUZ DEL CARMEN PEREZ OLAVARRIA (DIEMED),3305053,GI DEXMEDETOMIDINA 200MCG/2ML,110.0,110.0,110


In [ ]:
  # Write to CSV file
PurchasedOrders.to_csv('PurchasedOrders 2018-2019.csv', index=False)

### IncompleteOrders

In [99]:
IncompleteOrders = IncompleteOrders[IncompleteOrders['MedCode'].isin(PatentDrugs['MedCode'])].reset_index(drop=True)
IncompleteOrders

,SupplierName,OrderNumber,OrderDate,RequiredDeliveryDate,RequestNumber,RequestDate,CostCenter,MedCode,MedDescription,InvoiceNumber,ReceivedDate,AmountRequested,AmountPurchased,AmountReceived,OrderPercentageFulfillment
0,CASA MARZAM S.A. DE C.V.,274475.0,2018-08-02,2018-08-03,0.0,NaN,MEDICAMENTOS,3301093,DETRUSITOL 2MG C/14 TAB(TOLTERIDONA),NaN,NaN,0.0,2.0,0,0.0
1,CASA MARZAM S.A. DE C.V.,274475.0,2018-08-02,2018-08-03,0.0,NaN,MEDICAMENTOS,3301789,DOLO NEUROBION 50MG C/20 GRAGEAS (COMPLEJO B/D...,NaN,NaN,0.0,2.0,0,0.0
2,CASA MARZAM S.A. DE C.V.,274475.0,2018-08-02,2018-08-03,0.0,NaN,MEDICAMENTOS,3500493,FERRANINA FOL C/30 GRAG(HIERRO POLIMALTOSADO/A...,NaN,NaN,0.0,1.0,0,0.0
3,CASA MARZAM S.A. DE C.V.,274475.0,2018-08-02,2018-08-03,0.0,NaN,MEDICAMENTOS,3301938,FLONORM 200MG C/12 GRAG(RIFAXIMINA),NaN,NaN,0.0,2.0,0,0.0
4,CASA MARZAM S.A. DE C.V.,274475.0,2018-08-02,2018-08-03,0.0,NaN,MEDICAMENTOS,3301473,FLORATIL 200MG C/6 CAPS(SACCHAROMYCES BOULARDII),NaN,NaN,0.0,1.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2151,SOLUMEDIC S.A. DE C.V.,291258.0,2019-03-04,2019-03-08,0.0,NaN,MEDICAMENTOS,3300972,AGRASTAT (TIROFIBAN) 12.5MG/50ML FAM,NaN,NaN,0.0,1.0,0,0.0
2152,"VITASANITAS, S.A. DE C.V.",289309.0,2019-02-11,2019-02-18,60119.0,2019-02-11,BANCO SANGRE,3800003,ALBUMINA HUMANA AL 20% FCO 50ML,NaN,NaN,10.0,10.0,0,0.0
2153,"VITASANITAS, S.A. DE C.V.",290411.0,2019-02-22,2019-02-25,60340.0,2019-02-22,BANCO SANGRE,3800003,ALBUMINA HUMANA AL 20% FCO 50ML,NaN,NaN,10.0,10.0,0,0.0
2154,"VITASANITAS, S.A. DE C.V.",295721.0,2019-04-30,2019-05-02,0.0,NaN,BANCO SANGRE,3800003,ALBUMINA HUMANA AL 20% FCO 50ML,NaN,NaN,0.0,5.0,0,0.0


In [ ]:
  # Write to CSV file
IncompleteOrders.to_csv('IncompleteOrders 2018-2019.csv', index=False)

_________________

## Comparing PatientConsumption vs PatentDrugs

In [38]:
PatientConsumption = pd.read_csv('PatientConsumption 2018-2019.csv')
PatentDrugs        = pd.read_csv('PatentDrugs.csv')

  # Retrieve unique MedCode and MedDescription values in PatientConsumption
temp_df = PatientConsumption.loc[PatientConsumption['MedCode'].drop_duplicates().index, ['MedCode', 'MedDescription']].sort_values('MedCode').reset_index(drop=True)
temp_df

,MedCode,MedDescription
0,1000255.0,SIMILAXOL 78 100MG C/100
1,1000268.0,HALOPERIL 5MG/ML AMP IM (HALOPERIDOL)
2,1000280.0,FILATIL 300 MCG JER PRELL C/1 FILGRASTIM
3,1000294.0,IGEF (CELECOXIB) 200MG C/20 CAP
4,2400207.0,LENEDA 1%GELTB 30G
...,...,...
1676,3600131.0,ZYVOXAM TABLETAS 600MG C/10 (LINEZOLID)
1677,3600183.0,INNEFOL 50MG/4 ML AMP C/1 ACIDO FOLINICO
1678,3600195.0,EUTIROX 88MCG C/50LEVOTIROXINA SODICA
1679,3800003.0,ALBUMINA HUMANA AL 20% 50ML


In [39]:
  # Verifying that both PatentDrugs and temp_df have the same MedCode values
diff = PatentDrugs['MedCode'].sort_values().reset_index(drop=True) - temp_df['MedCode']
diff.value_counts()

0.0    1681
Name: MedCode, dtype: int64

___

## Add price columns to PatentDrugs diccionary

In [51]:
dicc_precio = pd.read_csv('diccionario_con_precio_v1_2.csv')
dicc_precio = PatentDrugs.merge(dicc_precio[['CODIGO','precio']], left_on='MedCode', right_on='CODIGO', how='left')
dicc_precio['MedCode'].equals(dicc_precio['CODIGO'])

True

In [57]:
dicc_precio = dicc_precio.drop(columns=['CODIGO']).rename(columns={'precio':'MedPriceMXN'})
dicc_precio.to_csv('PatentDrugs.csv', index=False)

In [58]:
PatentDrugs = pd.read_csv('PatentDrugs.csv')
PatentDrugs

,MedCode,MedDescription,MedPharmacon,MedFamily,MedPriceMXN
0,1000255,SIMILAXOL 78 100MG C/100,SIMILAXOL,Tracto alimentario y metabolismo,1092.0
1,1000268,HALOPERIL 5MG/ML AMP IM (HALOPERIDOL),HALOPERIL,Sistema nervioso,1162.0
2,1000280,FILATIL 300 MCG JER PRELL C/1 FILGRASTIM,FILATIL,Antineoplásicos e inmunomoduladores,699.0
3,1000294,IGEF (CELECOXIB) 200MG C/20 CAP,IGEF,Sistema musculoesquelético,1325.0
4,2400207,LENEDA 1%GELTB 30G,LENEDA,Dermatológicos,460.5
5,2900193,REFRESH TEARS GTS 15ML (CARBOXIMETILCELULOSA S...,REFRESH,Órganos de los sentidos,1480.0
6,2900217,"HIPOGLOS TUBO 110G (OXIDO DE ZINC/VITAMINA A,E)",HIPOGLOS,Dermatológicos,822.0
7,2900369,TRADERMA UNGUENTO 0.1%TACROLIMUS CAJA CON TUBO...,TRADERMA,Dermatológicos,599.0
8,2900517,ALBUMINA DE HUEVO 400GR OVOPROT,ALBUMINA,Sangre y órganos hematopoyéticos,NaN
9,2900557,ABINTRA POLVO 27G (SUPLEMENTO ALIMENTICIO),ABINTRA,Falla en Google,135.0
